In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Data_Wrangling").getOrCreate()

22/01/15 22:09:47 WARN Utils: Your hostname, Rose resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface enp3s0)
22/01/15 22:09:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/15 22:09:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
file_location = "./movie_data_part1.csv"
file_type = "csv"
infer_schema = "False"
first_row_is_header = "True"
delimiter = "|"

df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

In [5]:
df.printSchema()

root
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: string (nullable = true)



In [6]:
df.dtypes

[('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('vote_average', 'string')]

In [7]:
df.count()

43998

In [8]:
select_columns = ['id', 'budget', 'popularity', 'release_date', 'revenue', 'title']
df = df.select(select_columns)
df.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [9]:
df.select('id','budget','popularity','release_date','revenue','title').show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [10]:
df.show(25, False)

+-----+-------+------------------+------------+-------+--------------------------------------------------------+
|id   |budget |popularity        |release_date|revenue|title                                                   |
+-----+-------+------------------+------------+-------+--------------------------------------------------------+
|43000|0      |2.503             |1962-05-23  |0      |The Elusive Corporal                                    |
|43001|0      |5.51              |1962-11-12  |0      |Sundays and Cybele                                      |
|43002|0      |5.62              |1962-05-24  |0      |Lonely Are the Brave                                    |
|43003|0      |7.159             |1975-03-12  |0      |F for Fake                                              |
|43004|500000 |3.988             |1962-10-09  |0      |Long Day's Journey Into Night                           |
|43006|0      |3.194             |1962-03-09  |0      |My Geisha                                

In [11]:
from pyspark.sql.functions import *

In [12]:
df.filter((df['popularity']=='')|df['popularity'].isNull()|isnan(df['popularity'])).count()

215

In [13]:
df.select([count(when((col(c)=='') | 
                       col(c).isNull() | 
                       isnan(c), c)).alias(c) for c in df.columns]).show()

+---+------+----------+------------+-------+-----+
| id|budget|popularity|release_date|revenue|title|
+---+------+----------+------------+-------+-----+
|125|   125|       215|         221|    215|  304|
+---+------+----------+------------+-------+-----+



* Gom nhóm theo `title` sau đó đếm.

In [14]:
df.groupBy("title").count().show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|   The Corn Is Green|    1|
|Meet The Browns -...|    1|
|Morenita, El Esca...|    1|
| Father Takes a Wife|    1|
|The Werewolf of W...|    1|
|My Wife Is a Gang...|    1|
|Depeche Mode: Tou...|    1|
|  A Woman Is a Woman|    1|
|History Is Made a...|    1|
|      Colombian Love|    1|
|        Ace Attorney|    1|
|     Not Like Others|    1|
|40 Guns to Apache...|    1|
|          Middle Men|    1|
|         It's a Gift|    1|
|    La Vie de Bohème|    1|
|Rasputin: The Mad...|    1|
|The Ballad of Jac...|    1|
|         How to Deal|    1|
|             Freaked|    1|
+--------------------+-----+
only showing top 20 rows



* Gom nhóm theo `title` sau đó sắp xếp giảm dần dựa theo số lượng của nhóm.

In [20]:
df.groupBy('title').count().sort(desc('count')).show(truncate=False)

+-----------------------------+-----+
|title                        |count|
+-----------------------------+-----+
|null                         |304  |
|Cinderella                   |8    |
|The Three Musketeers         |8    |
|Les Misérables               |8    |
|Frankenstein                 |7    |
|Dracula                      |7    |
|A Christmas Carol            |7    |
|The Island                   |7    |
|Hamlet                       |7    |
|Crime and Punishment         |6    |
|Framed                       |6    |
|First Love                   |6    |
|Beauty and the Beast         |6    |
|Cleopatra                    |6    |
|The Hound of the Baskervilles|6    |
|The Lost World               |6    |
|Borderline                   |6    |
|Treasure Island              |6    |
|Blind Date                   |5    |
|Impulse                      |5    |
+-----------------------------+-----+
only showing top 20 rows



In [21]:
df_temp = df.filter((col('title') != "") &
                    (col('title').isNotNull()) &
                    (~isnan('title')))
df_temp.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [22]:
df_temp.groupBy('title').count().filter("`count` > 4")\
        .sort(col('count').desc()).show(truncate=False)

+-----------------------------+-----+
|title                        |count|
+-----------------------------+-----+
|Les Misérables               |8    |
|The Three Musketeers         |8    |
|Cinderella                   |8    |
|Dracula                      |7    |
|Frankenstein                 |7    |
|A Christmas Carol            |7    |
|The Island                   |7    |
|Hamlet                       |7    |
|Beauty and the Beast         |6    |
|Cleopatra                    |6    |
|First Love                   |6    |
|The Lost World               |6    |
|Treasure Island              |6    |
|Crime and Punishment         |6    |
|Framed                       |6    |
|Borderline                   |6    |
|The Hound of the Baskervilles|6    |
|Darling                      |5    |
|Dr. Jekyll and Mr. Hyde      |5    |
|Nightmare                    |5    |
+-----------------------------+-----+
only showing top 20 rows



In [31]:
df_temp.groupBy('title').count().filter("`count` >= 4")\
    .sort(desc('count')).count()

111

In [32]:
del df_temp

In [33]:
df.dtypes

[('id', 'string'),
 ('budget', 'string'),
 ('popularity', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('title', 'string')]

In [34]:
df = df.withColumn('budget', col('budget').cast('float'))
df.dtypes

[('id', 'string'),
 ('budget', 'float'),
 ('popularity', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('title', 'string')]

In [35]:
from pyspark.sql.types import *

In [37]:
int_vars = ['id']
float_vars = ['budget', 'popularity', 'revenue']
date_vars = ['release_date']

for c in int_vars:
    df = df.withColumn(c, df[c].cast(IntegerType()))
    
for c in float_vars:
    df = df.withColumn(c, df[c].cast(FloatType()))
    
for c in date_vars:
    df = df.withColumn(c, df[c].cast(DateType()))
    
df.dtypes

[('id', 'int'),
 ('budget', 'float'),
 ('popularity', 'float'),
 ('release_date', 'date'),
 ('revenue', 'float'),
 ('title', 'string')]

In [38]:
df.show()

+-----+---------+----------+------------+---------+--------------------+
|   id|   budget|popularity|release_date|  revenue|               title|
+-----+---------+----------+------------+---------+--------------------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|
|43007|      0.0|     2.689|  1962-10-31|      0.0|Period of Adjustment|
|43008|      0.0|     6.537|  1959-03-13|      0.0|    The Hanging Tree|
|43010|      0.0|     4.297|  1962-01-01|      0.0|Sherlock Holmes a...|
|43011|      0.0|     4.417|  1962-01-01|      0.0|  Sodom and Gomorrah|
|43012|7000000.0|     4.722|  1962-11-21|4000000.0|

In [40]:
df.describe().show()

+-------+------------------+--------------------+-----------------+-------------------+--------------------+
|summary|                id|              budget|       popularity|            revenue|               title|
+-------+------------------+--------------------+-----------------+-------------------+--------------------+
|  count|             43784|               43873|            43783|              43783|               43694|
|   mean|44502.304312077475|   3736901.834963167|5.295444259579189|  9697079.597382545|            Infinity|
| stddev|27189.646588626394|1.5871814952777326E7|6.168030519208252|5.687938449628811E7|                 NaN|
|    min|                 2|                 0.0|              0.6|                0.0|!Women Art Revolu...|
|    max|            100988|               3.8E8|            180.0|       2.78796518E9|       시크릿 Secret|
+-------+------------------+--------------------+-----------------+-------------------+--------------------+



In [47]:
df_temp = df.filter((col('budget') != 0) & 
                    (col('budget').isNotNull()) &
                    (~isnan('budget')))

median = df_temp.approxQuantile('budget', [0.5], 0.1)

median

[5000000.0]

In [48]:
df.agg(countDistinct(col("title")).alias("count")).show()

+-----+
|count|
+-----+
|41138|
+-----+



In [49]:
df.select('title').distinct().show(10,False)

+---------------------------------------------+
|title                                        |
+---------------------------------------------+
|The Corn Is Green                            |
|Meet The Browns - The Play                   |
|Morenita, El Escandalo                       |
|Father Takes a Wife                          |
|The Werewolf of Washington                   |
|My Wife Is a Gangster                        |
|Depeche Mode: Touring the Angel Live in Milan|
|A Woman Is a Woman                           |
|History Is Made at Night                     |
|Colombian Love                               |
+---------------------------------------------+
only showing top 10 rows



In [50]:
df_temp=df.withColumn('release_year',year('release_date'))
df_temp.show()

+-----+---------+----------+------------+---------+--------------------+------------+
|   id|   budget|popularity|release_date|  revenue|               title|release_year|
+-----+---------+----------+------------+---------+--------------------+------------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|        1962|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|        1962|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|        1962|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|        1975|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|        1962|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|        1962|
|43007|      0.0|     2.689|  1962-10-31|      0.0|Period of Adjustment|        1962|
|43008|      0.0|     6.537|  1959-03-13|      0.0|    The Hanging Tree|        1959|
|43010|      0.0|     4.297|  1962-01-01|      0.0|She